In [1]:
import torch
from ultralytics import YOLO

# Load the YOLOv8 model (best.pt)
model = YOLO('best.pt')

# Export the model to ONNX format
# imgsz specifies the input image size, which should match the size used during training
model.export(format='onnx', imgsz=416)  # 640 is a typical image size, but adjust if necessary


Ultralytics 8.3.81  Python-3.10.16 torch-2.3.1+cpu CPU (AMD Ryzen AI 9 HX 370 w/ Radeon 890M)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 7, 3549) (64.1 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success  1.2s, saved as 'best.onnx' (42.6 MB)

Export complete (1.6s)
Results saved to C:\Users\aup\EcoVision\model
Predict:         yolo predict task=detect model=best.onnx imgsz=416  
Validate:        yolo val task=detect model=best.onnx imgsz=416 data=/content/Recyclable-Items-3/data.yaml  
Visualize:       https://netron.app


'best.onnx'

In [2]:
from quark.onnx.quantization.config import Config, get_default_config
from quark.onnx.quantization.config.config import QuantizationConfig
from onnxruntime.quantization.calibrate import CalibrationMethod
from onnxruntime.quantization.quant_utils import QuantType, QuantFormat

# Use default quantization configuration
quant_config = get_default_config("XINT8")

# Define custom quantization configuration
# quant_config = QuantizationConfig(calibrate_method=PowerOfTwoMethod.MinMSE,   
#                                   activation_type=QuantType.QUInt8,
#                                   weight_type=QuantType.QInt8,
#                                   enable_npu_cnn=True,
#                                   extra_options={'ActivationSymmetric': True})

# Defines the quantization configuration for the whole model
config = Config(global_quant_config=quant_config)
print("The configuration of the quantization is {}".format(config))


[QUARK-INFO]: Custom Op compilation start.

[QUARK-INFO]: The custom_op already exists.

[QUARK-INFO]: Custom Op compilation already complete.


The configuration of the quantization is Config(global_quant_config=QuantizationConfig(calibrate_method=<PowerOfTwoMethod.MinMSE: 1>, quant_format=<QuantFormat.QDQ: 1>, activation_type=<QuantType.QUInt8: 1>, weight_type=<QuantType.QInt8: 0>, input_nodes=[], output_nodes=[], op_types_to_quantize=[], nodes_to_quantize=[], extra_op_types_to_quantize=[], nodes_to_exclude=[], specific_tensor_precision=False, execution_providers=['CPUExecutionProvider'], per_channel=False, reduce_range=False, optimize_model=True, use_dynamic_quant=False, use_external_data_format=False, convert_fp16_to_fp32=False, convert_nchw_to_nhwc=False, include_sq=False, include_cle=False, include_auto_mp=False, include_fast_ft=False, enable_npu_cnn=True, enable_npu_transformer=False, debug_mode=False, print_summary=True, ignore_warnings=True, log_severity_level=1, extra_options={'ActivationSymmetric': True}))


c:\ProgramData\anaconda3\envs\ryzen-ai-1.3.1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from quark.onnx import ModelQuantizer
# Create an ONNX Quantizer
quantizer = ModelQuantizer(config)

# Quantize the ONNX model
quant_model = quantizer.quantize_model(model_input = "best.onnx", 
                                       model_output = "best1quantized.onnx", 
                                       calibration_data_path = None)


[QUARK-INFO]: The input ONNX model best.onnx can create InferenceSession successfully

[QUARK-ERROR]: A calibration data reader is required for static quantization, but none was provided. Please provide a calibration data reader, or alternatively enable random data generation for calibration by setting `config.global_quant_config.extra_options["UseRandomData"]` to `True`.

[QUARK-ERROR]: A calibration data reader is required for static quantization, but none was provided. Please provide a calibration data reader, or alternatively enable random data generation for calibration by setting `config.global_quant_config.extra_options["UseRandomData"]` to `True`.


[QUARK_INFO]: Time information:
2025-03-02 00:50:00.507664
[QUARK_INFO]: OS and CPU information:
                                        system --- Windows
                                          node --- aup-strix7
                                       release --- 10
                                       version --- 10.0.26100
                                       machine --- AMD64
                                     processor --- AMD64 Family 26 Model 36 Stepping 0, AuthenticAMD
[QUARK_INFO]: Tools version information:
                                        python --- 3.10.16
                                          onnx --- 1.16.1
                                   onnxruntime --- 1.19.2
                                    quark.onnx --- 0.6.0+dba9ca364
[QUARK_INFO]: Quantized Configuration information:
                                   model_input --- best.onnx
                                  model_output --- best1quantized.onnx
                       calibration_data_re

ValueError: A calibration data reader is required for static quantization, but none was provided. Please provide a calibration data reader, or alternatively enable random data generation for calibration by setting `config.global_quant_config.extra_options["UseRandomData"]` to `True`.